# Clustering solutions using LDA #

In [32]:
# Input
from db import PythonProblems
import io

# Helpers
import numpy as np
from collections import Counter
from tqdm import tqdm
import random

# Preprocessing
import tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Learning
from sklearn.decomposition import LatentDirichletAllocation

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

### Connecting to database

In [2]:
db = PythonProblems('python.sqlite')

In [3]:
removed_itens = ['NEWLINE', 'STRING', 'ENDMARKER', 'NUMBER', 'INDENT', 'DEDENT', "NL", 'COMMENT', 'ERRORTOKEN']
allowed_itens = ['NAME', 'OP']
cursor = db.conn.cursor()
docs = []
errors = []

# lendo os dados
cursor.execute("""
SELECT * FROM solution;
""")


for idx, linha in enumerate(cursor.fetchall()):
    file = io.StringIO(linha[1])
    doc = []
    try:
        for item in tokenize.generate_tokens(file.readline):
            if tokenize.tok_name[item[0]] not in removed_itens:
                if tokenize.tok_name[item[0]] in allowed_itens:
                    doc.append(item[1])
                else:
                    print("%s %s" % (tokenize.tok_name[item[0]], item[1]))
    except (IndentationError, tokenize.TokenError):
        errors.append("Please, fix solution %d before continuing" % (idx+1))
        
    docs.append(' '.join(doc))

print("Got %d documents" %(idx+1))

if not errors:
    print("Success in parsing all documents! You may go on!")
else:
    for item in errors:
        print(item)

Got 758 documents
Success in parsing all documents! You may go on!


### Preprocessing solutions into bag of words ###

In [33]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
#vectorizer = CountVectorizer(analyzer = "word",   \
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             #max_features = 26d,
                             binary=False,
                             min_df=2
                            ) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.

train_data_features = vectorizer.fit_transform(docs)

# Numpy arrays are easy to work with, so convert the result to an 
# array
# Document-term matrix
#train_data_features = train_data_features.toarray()

# Cálculo da LDA #

### Cálculo dos modelos e da métrica *perplexity* ###

In [34]:
num_topics = list(range(2, 100))

In [ ]:
output_notebook()

hover = HoverTool(tooltips=[
    ("Model", "$x{0}"),
    ("Perplexity", "$y"),
])


models = {}
x_values = []
y_values = []

for k in tqdm(num_topics):
  # Build a Latent Dirichlet Allocation Model
  model = LatentDirichletAllocation(n_components=k, max_iter=10, learning_method='batch')
  document_topic = model.fit_transform(train_data_features)
  models[k] = {
      'model': model,
      'topic_word': model.components_,
      'document_topic': document_topic,
      'perplexity': model.perplexity(train_data_features)
  }
  
  x_values.append(k)
  y_values.append(models[k]["perplexity"])

source = {'x_values': x_values, 'y_values': y_values}

opts = dict(plot_width=900, plot_height=500, min_border=0, title="Perplexity")
plot = figure(**opts)
plot.add_tools(hover)

l = plot.line(x='x_values', y='y_values', color="#2222aa", line_width=2, source=source)
c = plot.circle(x='x_values', y='y_values', color="#2222aa", fill_color="white", size=8, source=source)
h = show(plot, notebook_handle=True)

Loading BokehJS ...

 32%|███▏      | 31/98 [00:47<01:41,  1.52s/it]

### Verificando em * random sample solutions* ###

In [ ]:
# Escolhido tópico k
k = 12
lda_model = models[k]['model']
document_topic = models[k]['document_topic']
topic_word = models[k]['topic_word']

In [ ]:
# Verificando 5 soluções de maneira aleatória
N = 5

cursor.execute("""
SELECT * FROM solution;
""")

solutions = list(cursor.fetchall())
random.shuffle(list(range(len(solutions))))

for i in range(N):
    text = solutions[i][1]
    x = lda_model.transform(vectorizer.transform([text]))[0]
    print(text)
    #print(x)
    print("Belong to topics with probabilities:")
    topics = np.argsort(x)[:-6:-1]
    probs = x[topics]
    print("t: %s" %(topics+1))
    print("P(t|d): %s" %probs)

    print('\n')

In [ ]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, train_data_features, vectorizer, mds='tsne', sort_topics=False)
panel

#### Verificando distribuição de documentos por tópico ####

In [ ]:
row_sums = document_topic.sum(axis=1)
document_topic_norm = document_topic / row_sums[:, np.newaxis]
document_topic_norm.shape

Considerando somente pertinência máxima

In [ ]:
Counter(np.argmax(document_topic, axis=1))

Considerando probabilidade

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = sns.heatmap(document_topic_norm)
plt.show()

In [ ]:
def print_top_words(topic_word, feature_names, n_top_words):
    for topic_idx, topic in enumerate(topic_word):
        message = "Topic #%d: " % (topic_idx+1)
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print("\nTopics in LDA model:")
feature_names = vectorizer.get_feature_names()
print_top_words(topic_word, feature_names, 10)